# Model Test

⚠ Work In Progress ⚠

In [ ]:
import os
import numpy as np
import json
import pandas as pd
from birdnetlib.analyzer import Analyzer
from birdnetlib.batch import DirectoryAnalyzer
import copy         

# Validation Results
Introduction of all the variables to start the validation process.

This involves:

1. Evaluation and extraction of the predicted segments in validation set
2. Threshold computation from validation results

In [21]:
# check 1_segments_extraction for true_segments_test
DATASET_NAME = 'NEW_DATASET'
DATASET_PATH = f'E:/Giacomo/Tovanella/{DATASET_NAME}'
MODEL_NAME = "angelo"
with open(f'utils/{DATASET_NAME}/true_segments.json') as f:
    true_segments = json.load(f)

with open(f'utils/WABAD/true_segments_wabad.json') as f:
    true_segments_wabad = json.load(f)

In [22]:
# !!! remember to remove non-species class in CustomClassifierLabels
labels_path = f"classifiers\official\{MODEL_NAME}\CustomClassifier_Labels.txt"
labels_path_1 = f"classifiers\official\{MODEL_NAME}\CustomClassifier_Labels_1.txt"
with open(labels_path, 'r') as infile, open(labels_path_1, 'w') as outfile:
    for line in infile:
        line = line.strip()  # Remove leading/trailing whitespace
        if "_" in line:
            outfile.write(line + '\n')

os.rename(labels_path, labels_path + "_old")
os.rename(labels_path_1, labels_path)

FileExistsError: [WinError 183] Impossibile creare un file, se il file esiste già: 'classifiers\\official\\angelo\\CustomClassifier_Labels.txt' -> 'classifiers\\official\\angelo\\CustomClassifier_Labels.txt_old'

In [ ]:
class BirdAnalyzer:
    def __init__(self, model_name, dataset_path, min_conf=0.1):
        self.model_name = model_name
        self.dataset_path = dataset_path
        self.min_conf = min_conf
        self.complete_pred_segments = {}
        
        self.model_path = f"classifiers/official/{model_name}/CustomClassifier.tflite"
        self.labels_path = f"classifiers/official/{model_name}/CustomClassifier_Labels.txt"
        self.test_path = os.path.join(dataset_path, 'test')
        self.valid_path = os.path.join(dataset_path, 'valid')
        
        self.analyzer = Analyzer(
            classifier_labels_path=self.labels_path, 
            classifier_model_path=self.model_path
        )
    
    def _on_analyze_complete(self, recording):
        audio_name = recording.path.split('\\')[-1]
        from_wabad = len(audio_name.split('_')) == 5

        if from_wabad:
            site, date, hour, segm_sec1, segm_sec2 = audio_name.split('_')
            audio_name = "_".join([site, date, hour]) + ".WAV"
        else:
            date, hour, segm_sec1, segm_sec2 = audio_name.split('_')
            audio_name = "_".join([date, hour]) + ".WAV"

        segm_sec2 = segm_sec2.split('.')[0]
        segm_id = "_".join([segm_sec1, segm_sec2])
        
        if audio_name not in self.complete_pred_segments:
            self.complete_pred_segments[audio_name] = {}
        
        self.complete_pred_segments[audio_name][segm_id] = {
            detection["label"]: detection["confidence"] for detection in recording.detections
        }
        
        print("Analyzing ", recording.path)
    
    def on_error(self, recording, error):
        print(f"An exception occurred: {error}")
        print(recording.path)
    
    def process_data_set(self, data_set_type="valid"):
        if data_set_type not in ["valid", "test"]:
            raise ValueError("data_set_type must be 'valid' or 'test'")

        if data_set_type == "valid":
            data_path = self.valid_path
            output_filename = "valid_pred_segments.json"
        else:
            data_path = self.test_path
            output_filename = "test_pred_segments.json"

        self.complete_pred_segments = {}
        
        for folder in os.listdir(data_path):
            directory = os.path.join(data_path, folder)
            print(f"Starting Watcher for {data_set_type} set, folder: {folder}")

            batch = DirectoryAnalyzer(
                directory,
                analyzers=[self.analyzer],
                min_conf=self.min_conf,
            )
            batch.on_analyze_complete = self._on_analyze_complete
            batch.on_error = self.on_error
            batch.process()

        pred_segments = copy.deepcopy(self.complete_pred_segments)
        output_path = f"classifiers/official/{self.model_name}/{output_filename}"
        with open(output_path, "w") as f:
            json.dump(pred_segments, f)

        print(f"{data_set_type.capitalize()} predictions saved to: {output_path}")
        return pred_segments


## Evaluation and Extraction from Validation

In [24]:
bird_analyzer = BirdAnalyzer(MODEL_NAME, DATASET_PATH, 0.1)
valid_done = False
test_done = False

load_custom_models
Custom model loaded.
loading custom classifier labels
Labels loaded.
load model False
Model loaded.
Labels loaded.
load_species_list_model
Meta model loaded.


In [6]:
if not valid_done:
    valid_pred_segments = bird_analyzer.process_data_set("valid")
valid_done = True

Starting Watcher for valid set, folder: Certhia familiaris_Eurasian Treecreeper
read_audio_data
read_audio_data: complete, read  1 chunks.
analyze_recording 20190603_030000_541_5.wav
Analyzing  E:\Giacomo\Tovanella\NEW_DATASET_1\valid\Certhia familiaris_Eurasian Treecreeper\20190603_030000_541_5.wav
read_audio_data
read_audio_data: complete, read  1 chunks.
analyze_recording 20190603_030000_574_5.wav
Analyzing  E:\Giacomo\Tovanella\NEW_DATASET_1\valid\Certhia familiaris_Eurasian Treecreeper\20190603_030000_574_5.wav
read_audio_data
read_audio_data: complete, read  1 chunks.
analyze_recording 20190603_170000_472_5.wav
Analyzing  E:\Giacomo\Tovanella\NEW_DATASET_1\valid\Certhia familiaris_Eurasian Treecreeper\20190603_170000_472_5.wav
read_audio_data
read_audio_data: complete, read  1 chunks.
analyze_recording 20190603_170000_483_0.wav
Analyzing  E:\Giacomo\Tovanella\NEW_DATASET_1\valid\Certhia familiaris_Eurasian Treecreeper\20190603_170000_483_0.wav
read_audio_data
read_audio_data: com

In [7]:
# LOAD
with open(f"classifiers/official/{MODEL_NAME}/valid_pred_segments.json") as f:
    valid_pred_segments = json.load(f)

## Thresold Computation from Validation results

Da ricordare che in Validation sono presenti WABAD e originali. Per calcolare le soglie di confidenza, devo capire se la predizione del modello è corretta e quale soglia associargli.

In conf_scores.json vado a inserire tutti i confidence scores relativi ad una specie e gli assegno True se il modello ha predetto correttamente, False altrimenti.

In [8]:
valid_species_list = os.listdir(f"{DATASET_PATH}/valid")

In [9]:
conf_scores = { species: [] for species in valid_species_list }
for audio in valid_pred_segments.keys():                            # {'20190603_030000.WAV': {'541_5': {'species_B': 0.256, '<species_A>': 0.195,
    for segm, species in valid_pred_segments[audio].items():        # {'541_5': {'species_B': 0.256, '<species_A>': 0.195,
        for sp in species:
            if sp not in valid_species_list:
                continue
            score = valid_pred_segments[audio][segm][sp]
            if score <= 0.15:
                continue
            is_wabad = len(audio.split("_")) == 3                   # <BAM>_<20151130>_<091318.WAV>
            if is_wabad:
                is_correct = sp in true_segments_wabad[audio][segm]
            else:
                is_correct = sp in true_segments[audio][segm]
            conf_scores[sp].append([score, is_correct])

In [10]:
with open(f"classifiers/official/{MODEL_NAME}/conf_scores.json", "w") as f:
    json.dump(conf_scores, f)

In [11]:
from sklearn.metrics import precision_recall_fscore_support

best_threshold_by_species = {}
default_threshold = 0.15
for specie, data in conf_scores.items():
    if len(data) <= 10:
        best_threshold_by_species[specie] = default_threshold
        continue
    scores = np.array([x[0] for x in data])
    labels = np.array([x[1] for x in data])

    best_threshold = 0
    best_f1 = 0
    thresholds = np.linspace(scores.min(), scores.max(), 50)  # 100 soglie testate

    for threshold in thresholds:
        preds = scores >= threshold
        precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="binary", zero_division=0)
        
        if f1 > best_f1:
            best_f1 = f1
            best_threshold = threshold
    best_threshold_by_species[specie] = best_threshold
    print(f"Optimal threshold for {specie}: {best_threshold:.3f} with f1 score: {best_f1:.3f}")

Optimal threshold for Dryocopus martius_Black Woodpecker: 0.000 with f1 score: 0.000
Optimal threshold for Erithacus rubecula_European Robin: 0.000 with f1 score: 0.000
Optimal threshold for Fringilla coelebs_Common Chaffinch: 0.495 with f1 score: 0.286
Optimal threshold for Glaucidium passerinum_Eurasian Pygmy-Owl: 0.000 with f1 score: 0.000
Optimal threshold for Lophophanes cristatus_Crested Tit: 0.252 with f1 score: 0.002
Optimal threshold for Periparus ater_Coal Tit: 0.237 with f1 score: 0.508
Optimal threshold for Phylloscopus collybita_Common Chiffchaff: 0.000 with f1 score: 0.000
Optimal threshold for Phylloscopus trochilus_Willow Warbler: 0.000 with f1 score: 0.000
Optimal threshold for Prunella modularis_Dunnock: 0.220 with f1 score: 0.043
Optimal threshold for Pyrrhula pyrrhula_Eurasian Bullfinch: 0.236 with f1 score: 0.014
Optimal threshold for Regulus ignicapilla_Common Firecrest: 0.000 with f1 score: 0.000
Optimal threshold for Spinus spinus_Eurasian Siskin: 0.000 with f1 

In [12]:
# save best_threshold_by_species
with open(f"classifiers/official/{MODEL_NAME}/best_threshold_by_species.json", "w") as f:
    json.dump(best_threshold_by_species, f)

# Test Results

In [ ]:
if not test_done:
    test_pred_segments = bird_analyzer.process_data_set("test")
test_done = True

Starting Watcher for test set, folder: Certhia familiaris_Eurasian Treecreeper
read_audio_data
read_audio_data: complete, read  1 chunks.
analyze_recording 20190608_070000_63.wav
An exception occurred: not enough values to unpack (expected 4, got 3)
E:\Giacomo\Tovanella\NEW_DATASET\test\Certhia familiaris_Eurasian Treecreeper\20190608_070000_63.wav
read_audio_data
read_audio_data: complete, read  1 chunks.
analyze_recording 20190608_070000_82.wav
An exception occurred: not enough values to unpack (expected 4, got 3)
E:\Giacomo\Tovanella\NEW_DATASET\test\Certhia familiaris_Eurasian Treecreeper\20190608_070000_82.wav
read_audio_data
read_audio_data: complete, read  1 chunks.
analyze_recording 20190608_070000_83.wav
An exception occurred: not enough values to unpack (expected 4, got 3)
E:\Giacomo\Tovanella\NEW_DATASET\test\Certhia familiaris_Eurasian Treecreeper\20190608_070000_83.wav
read_audio_data
read_audio_data: complete, read  1 chunks.
analyze_recording 20190608_070000_88.wav
An ex

# Test Evaluation

In [14]:
def filter_predictions_by_threshold(pred_segments, best_threshold_by_species):
    filtered_pred_segments = {}
    for audio, segments in pred_segments.items():
        filtered_pred_segments[audio] = {}
        for segm_id, predictions in segments.items():
            species_to_save = [
                label for label, conf_score in predictions.items()
                if conf_score > best_threshold_by_species.get(label, 0)
            ]
            filtered_pred_segments[audio][segm_id] = species_to_save
    return filtered_pred_segments


In [15]:
filtered_pred_segments = filter_predictions_by_threshold(test_pred_segments, best_threshold_by_species)

In [16]:
test_species_list = os.listdir(f"{DATASET_PATH}/test")

In [17]:
# extract only test segments from true_segments
test_true_segments = {}
for audio in filtered_pred_segments.keys():                         # {'20190608_070000.WAV': {'189_0': ['Poecile palustris_Marsh Tit'],
    test_true_segments[audio] = {}                                  # {'20190608_070000.WAV' : {}}
    for key, values in filtered_pred_segments[audio].items():       #  {'189_0': ['Poecile palustris_Marsh Tit'],
        test_true_segments[audio][key] = [value for value in true_segments[audio][key] if value in test_species_list]  # {'20190608_070000.WAV': {'189_0': ['Poecile palustris_Marsh Tit'],

In [18]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()
mlb.fit([list(test_species_list)])

len(mlb.classes_)

17

In [19]:
y_pred = []
y_true = []
for audio in test_pred_segments:
    # sort in increasing order
    sortable_true_segments = { str(key): value for key, value in test_true_segments[audio].items() }
    sortable_pred_segments = { str(key): value for key, value in test_pred_segments[audio].items() }
    sorted_true_segments = dict(sorted(sortable_true_segments.items()))
    sorted_pred_segments = dict(sorted(sortable_pred_segments.items()))
    y_true.append(mlb.transform(sorted_true_segments.values()))     # apply transform on every label of every segment
    y_pred.append(mlb.transform(sorted_pred_segments.values()))     # apply transform on every label of every segment

y_true = np.vstack(y_true)
y_pred = np.vstack(y_pred)

c:\Users\giaco\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\preprocessing\_label.py:878: UserWarning: unknown class(es) ['Phylloscopus trochilus_Willow Warbler'] will be ignored
  warnings.warn(
c:\Users\giaco\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\preprocessing\_label.py:878: UserWarning: unknown class(es) ['Prunella modularis_Dunnock'] will be ignored
  warnings.warn(
c:\Users\giaco\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\preprocessing\_label.py:878: UserWarning: unknown class(es) ['Phylloscopus collybita_Common Chiffchaff', 'Poecile palustris_Marsh Tit', 'Prunella modularis_Dunnock'] will be ignored
  warnings.warn(
c:\Users\giaco\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\preprocessing\_label.py:878: UserWarning: unknown class(es) ['Poecile palustris_Marsh Tit', 'Prunella modularis_Dunnock'] will be ignored
  warnings.warn(
c:\Users\giaco\AppData\Local\Programs\Python\Python310\lib\site

In [20]:
from sklearn.metrics import classification_report

report = classification_report(y_true, y_pred, target_names=mlb.classes_, zero_division=0, output_dict=True)
with open(f"classifiers/official/{MODEL_NAME}/classification_report.json", "w") as f:
    json.dump(report, f)

report_df = pd.DataFrame(report).T
report_df

,precision,recall,f1-score,support
Certhia familiaris_Eurasian Treecreeper,0.000000,0.000000,0.000000,76.0
Dendrocopos major_Great Spotted Woodpecker,0.000000,0.000000,0.000000,27.0
Dryocopus martius_Black Woodpecker,0.000000,0.000000,0.000000,24.0
Erithacus rubecula_European Robin,0.000000,0.000000,0.000000,194.0
Fringilla coelebs_Common Chaffinch,0.000000,0.000000,0.000000,2702.0
Glaucidium passerinum_Eurasian Pygmy-Owl,0.000000,0.000000,0.000000,27.0
Lophophanes cristatus_Crested Tit,0.014286,0.604651,0.027912,43.0
Loxia curvirostra_Common Crossbill,0.000000,0.000000,0.000000,60.0
Muscicapa striata_Spotted Flycatcher,0.000000,0.000000,0.000000,159.0
Periparus ater_Coal Tit,0.000000,0.000000,0.000000,151.0
